In [8]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from  matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential,load_model,save_model
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.layers import Conv2D, MaxPooling2D
import matplotlib.image as mpimg
%matplotlib inline

In [2]:
batch_size = 32
img_height = 299
img_width = 299
image_size = (img_height, img_width)

In [3]:
data_dir1 = "/content/drive/MyDrive/Colab Notebooks/Major/Data/train"
data_dir2 = "/content/drive/MyDrive/Colab Notebooks/Major/Data/val"
data_dir3 = "/content/drive/MyDrive/Colab Notebooks/Major/Data/test"
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir1,
  validation_split=None,
  shuffle=True,
  image_size=(img_height, img_width),
  subset=None,
  seed=123,
  batch_size=batch_size,
  label_mode='categorical'
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir2,
  validation_split=None,
  shuffle=True,
  image_size=(img_height, img_width),
  subset=None,
  seed=123,
  batch_size=batch_size,
  label_mode='categorical'
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir3,
  validation_split=None,
  shuffle=True,
  image_size=(img_height, img_width),
  subset=None,
  seed=123,
  batch_size=batch_size,
  label_mode='categorical'
)

Found 11447 files belonging to 5 classes.
Found 2445 files belonging to 5 classes.
Found 2445 files belonging to 5 classes.


In [6]:
from keras.applications.mobilenet import MobileNet

In [9]:
model = Sequential()
base_model = MobileNet(input_shape=(img_width, img_height, 3), include_top = False, weights = None)
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(activation = 'relu', units = 128))
model.add(Dropout(0.25))
model.add(Dense(activation = 'relu', units = 64))
model.add(Dropout(0.2))
model.add(Dense(5, activation = 'softmax'))

In [12]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.build(input_shape=(1, img_width, img_height, 3))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_1 (ModuleWrap (1, 9, 9, 1024)           3228864   
_________________________________________________________________
global_average_pooling2d (Gl (1, 1024)                 0         
_________________________________________________________________
dropout (Dropout)            (1, 1024)                 0         
_________________________________________________________________
dense (Dense)                (1, 128)                  131200    
_________________________________________________________________
dropout_1 (Dropout)          (1, 128)                  0         
_________________________________________________________________
dense_1 (Dense)              (1, 64)                   8256      
_________________________________________________________________
dropout_2 (Dropout)          (1, 64)                  

In [13]:
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=3)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)

callbacks_list = [ early, learning_rate_reduction]

In [15]:
# without callbacks
model.fit(
    train_ds ,
    epochs=30, 
    validation_data=val_ds, 
    callbacks=callbacks_list
)
# fcking slow

Epoch 1/30
358/358 [==============================] - 4932s 14s/step - loss: 0.7231 - accuracy: 0.7182 - val_loss: 2.6332 - val_accuracy: 0.2147
Epoch 2/30
358/358 [==============================] - 3443s 10s/step - loss: 0.3830 - accuracy: 0.8660 - val_loss: 2.5730 - val_accuracy: 0.4180
Epoch 3/30
358/358 [==============================] - 3430s 10s/step - loss: 0.2843 - accuracy: 0.9070 - val_loss: 2.7212 - val_accuracy: 0.5513
Epoch 4/30
 50/358 [===>..........................] - ETA: 46:53 - loss: 0.2315 - accuracy: 0.9237

KeyboardInterrupt: ignored